<a href="https://colab.research.google.com/github/trajinthan/Creditcard-Default-prediction/blob/master/randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd   #to use read_csv(), crosstab()
import numpy as np    #to use exponentiation : exp()
import statsmodels.api as sm  #to use the Logit()
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline

In [0]:
#import the dataset
traindata=pd.read_csv('credit_card_default_train.csv',sep=",")
testdata=pd.read_csv('credit_card_default_test.csv',sep=",")

In [0]:
traindata=traindata.replace(['M','F'], [0,1]) 
testdata=testdata.replace(['M','F'], [0,1]) 

In [0]:
traindata['EDUCATION_STATUS']=traindata['EDUCATION_STATUS'].replace(['Graduate','High School','Other'], [3,2,1]) 
testdata['EDUCATION_STATUS']=testdata['EDUCATION_STATUS'].replace(['Graduate','High School','Other'], [3,2,1]) 

In [0]:
traindata['MARITAL_STATUS']=traindata['MARITAL_STATUS'].replace(['Single','Other'], [1,2]) 
testdata['MARITAL_STATUS']=testdata['MARITAL_STATUS'].replace(['Single','Other'], [1,2]) 

In [0]:
traindata['AGE']=traindata['AGE'].replace(['Less than 30','31-45','46-65','More than 65'], [4,3,2,1]) 
testdata['AGE']=testdata['AGE'].replace(['Less than 30','31-45','46-65','More than 65'], [4,3,2,1]) 


In [0]:
traindata['Balance_Limit_V1'] = traindata['Balance_Limit_V1'].map(lambda x : float(x[0:-1])*1000 if (x[-1]=='K') else float(x[0:-1])*1000000)
testdata['Balance_Limit_V1'] = testdata['Balance_Limit_V1'].map(lambda x : float(x[0:-1])*1000 if (x[-1]=='K') else float(x[0:-1])*1000000)

In [0]:
# testdata[testdata.columns[12:24]]=testdata[testdata.columns[12:24]]/traindata.mean()[traindata.columns[12:24]]
# traindata[traindata.columns[12:24]]=traindata[traindata.columns[12:24]]/traindata.mean()[traindata.columns[12:24]]


In [0]:
# testdata['Balance_Limit_V1']=testdata['Balance_Limit_V1']/traindata.mean()['Balance_Limit_V1']
# traindata['Balance_Limit_V1']=traindata['Balance_Limit_V1']/traindata.mean()['Balance_Limit_V1']


In [0]:
# output = 'NEXT_MONTH_DEFAULT'

# # Let's do a little EDA
# cols = [ f for f in traindata.columns if traindata.dtypes[ f ] != "object"]

# cols.remove( output )

# f = pd.melt( traindata, id_vars=output, value_vars=cols)
# g = sns.FacetGrid( f, hue=output, col="variable", col_wrap=5, sharex=False, sharey=False )
# g = g.map( sns.distplot, "value", kde=True).add_legend()


In [0]:
traindata['BIAS']=1
testdata['BIAS']=1

In [0]:
independentVar= ['Balance_Limit_V1',
                'Gender',
                  # 'EDUCATION_STATUS',
                  'MARITAL_STATUS',
                  # 'AGE',
                 'PAY_JULY','PAY_AUG','PAY_SEP','PAY_OCT','PAY_NOV','PAY_DEC',
                 'DUE_AMT_JULY','DUE_AMT_AUG','DUE_AMT_SEP','DUE_AMT_OCT','DUE_AMT_NOV','DUE_AMT_DEC',
                'PAID_AMT_JULY','PAID_AMT_AUG','PAID_AMT_SEP','PAID_AMT_OCT','PAID_AMT_NOV','PAID_AMT_DEC',
                #  'BIAS'
                 ]

X_train, y_train,X_test =traindata[independentVar],traindata['NEXT_MONTH_DEFAULT'],testdata[independentVar]

X_train, y_train =traindata[independentVar],traindata['NEXT_MONTH_DEFAULT']

from sklearn.model_selection import train_test_split
X_train,Xtest, y_train,ytest = train_test_split(X_train,y_train,test_size=0.0001)

from sklearn.preprocessing import StandardScaler
scX = StandardScaler()
X_train = scX.fit_transform( X_train )
X_test = scX.transform( X_test )




# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler
 

# # define undersampling strategy
# under = RandomUnderSampler(sampling_strategy=0.4)
# # fit and apply the transform
# X_train_res, y_train_res = under.fit_resample(X_train, y_train)

# sm = SMOTE(random_state=2,sampling_strategy=0.5)
# X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())





In [0]:

from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
# classifier = RandomForestClassifier(n_estimators=10)
# classifier.fit( X_train, y_train)


classifier= AdaBoostClassifier(random_state=41,learning_rate=0.3,n_estimators=5)
classifier.fit(X_train, y_train)

# classifier = DecisionTreeClassifier(random_state=41,max_leaf_nodes=5)
# classifier.fit(X_train,y_train)

# classifier = GradientBoostingClassifier(random_state=41,n_estimators=10,max_depth=4)
# classifier.fit(X_train,y_train)





AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.3,
                   n_estimators=6, random_state=41)

In [0]:
y_pred = classifier.predict(X_test)
y_pred1 = classifier.predict(Xtest)
y=[]
for i in y_pred:
  y.append([i])
ydata=pd.DataFrame(y,columns=['NEXT_MONTH_DEFAULT'])
ydata['Client_ID']=testdata['Client_ID']
ydata.to_csv('output.csv')
print (ydata.groupby(['NEXT_MONTH_DEFAULT']).count())




from sklearn import metrics
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(ytest, y_pred1)
print(confusion_matrix)

                    Client_ID
NEXT_MONTH_DEFAULT           
0                        5387
1                         613
[[2 0]
 [1 0]]


In [0]:
from sklearn.metrics import classification_report
print(classification_report(ytest, y_pred1))


              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       0.00      0.00      0.00         1

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
